# Cassandra 数据库工具包

> `Apache Cassandra®` 是一个广泛用于存储事务性应用程序数据的数据库。大型语言模型中函数和工具的引入为生成式 AI 应用中现有数据开辟了一些令人兴奋的用例。

> `Cassandra Database` 工具包使 AI 工程师能够有效地将代理与 Cassandra 数据集成，并提供以下功能：
> - 通过优化的查询实现快速数据访问。大多数查询应能在个位数毫秒或更少时间内运行。
> - 模式内省以增强 LLM 的推理能力。
> - 与各种 Cassandra 部署兼容，包括 Apache Cassandra®、DataStax Enterprise™ 和 DataStax Astra™。
> - 目前，该工具包仅限于 SELECT 查询和模式内省操作。（安全第一）

有关创建 Cassandra DB 代理的更多信息，请参阅 [CQL 代理使用指南](https://github.com/langchain-ai/langchain/blob/master/cookbook/cql_agent.ipynb)。

## 快速入门
- 安装 `cassio` 库
- 为您连接的 Cassandra 数据库设置环境变量
- 初始化 `CassandraDatabase`
- 使用 `toolkit.get_tools()` 将工具传递给您的代理
- 坐下来看着它为您完成所有工作

## 工作原理

`Cassandra 查询语言 (CQL)` 是与 Cassandra 数据库交互的主要*以人为中心*的方式。虽然在生成查询时提供了一些灵活性，但它需要了解 Cassandra 数据建模的最佳实践。LLM 函数调用使代理能够进行推理，然后选择一个工具来满足请求。使用 LLM 的代理在选择合适的工具包或工具包链时，应使用特定于 Cassandra 的逻辑进行推理。这减少了当 LLM 被迫提供自上而下的解决方案时引入的随机性。您希望 LLM 完全不受限制地访问您的数据库吗？是的。可能不是。为了实现这一点，我们在构建代理问题时提供了一个提示：

您是一个 Apache Cassandra 专家查询分析机器人，具有以下功能和规则：
- 您将接收最终用户关于在数据库中查找特定数据的请求。
- 您将检查数据库的模式并创建查询路径。
- 您将为用户提供查找其所需数据的正确查询，并显示查询路径提供的步骤。
- 您将使用使用分区键和集群列查询 Apache Cassandra 的最佳实践。
- 避免在查询中使用 ALLOW FILTERING。
- 目标是找到一个查询路径，因此可能需要查询其他表才能得到最终答案。

以下是 JSON 格式的查询路径示例：

```json
 {
  "query_paths": [
    {
      "description": "Direct query to users table using email",
      "steps": [
        {
          "table": "user_credentials",
          "query":
             "SELECT userid FROM user_credentials WHERE email = 'example@example.com';"
        },
        {
          "table": "users",
          "query": "SELECT * FROM users WHERE userid = ?;"
        }
      ]
    }
  ]
}
```

## 提供的工具

### `cassandra_db_schema`
收集连接数据库或特定模式的所有模式信息。对代理确定操作至关重要。

### `cassandra_db_select_table_data`
从特定的 keyspace 和表中选择数据。代理可以传递参数以进行谓词过滤和限制返回记录的数量。

### `cassandra_db_query`
`cassandra_db_select_table_data` 的实验性替代方案，它接受一个由代理完全形成的查询字符串，而不是参数。*警告*：这可能导致非标准的查询，这些查询性能可能不佳（甚至无法工作）。它可能会在未来的版本中被移除。如果您发现了它的有趣之处，我们也想了解。谁知道呢！

## 环境设置

安装以下 Python 模块：

```bash
pip install ipykernel python-dotenv cassio langchain_openai langchain langchain-community langchainhub
```

### .env 文件
连接通过 `cassio` 并使用 `auto=True` 参数，笔记本使用 OpenAI。您应该相应地创建一个 `.env` 文件。

对于 Cassandra，设置：
```bash
CASSANDRA_CONTACT_POINTS
CASSANDRA_USERNAME
CASSANDRA_PASSWORD
CASSANDRA_KEYSPACE
```

对于 Astra，设置：
```bash
ASTRA_DB_APPLICATION_TOKEN
ASTRA_DB_DATABASE_ID
ASTRA_DB_KEYSPACE
```

例如：

```bash
# 连接到 Astra:
ASTRA_DB_DATABASE_ID=a1b2c3d4-...
ASTRA_DB_APPLICATION_TOKEN=AstraCS:...
ASTRA_DB_KEYSPACE=notebooks

# 同时设置
OPENAI_API_KEY=sk-....
```

（您也可以修改以下代码以直接使用 `cassio` 连接。）

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

In [2]:
# Import necessary libraries
import os

import cassio
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.agent_toolkits.cassandra_database.toolkit import (
    CassandraDatabaseToolkit,
)
from langchain_community.tools.cassandra_database.prompt import QUERY_PATH_PROMPT
from langchain_community.utilities.cassandra_database import CassandraDatabase
from langchain_openai import ChatOpenAI

## 连接到 Cassandra 数据库

In [3]:
cassio.init(auto=True)
session = cassio.config.resolve_session()
if not session:
    raise Exception(
        "Check environment configuration or manually configure cassio connection parameters"
    )

In [4]:
# Test data pep

session = cassio.config.resolve_session()

session.execute("""DROP KEYSPACE IF EXISTS langchain_agent_test; """)

session.execute(
    """
CREATE KEYSPACE if not exists langchain_agent_test 
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
"""
)

session.execute(
    """
    CREATE TABLE IF NOT EXISTS langchain_agent_test.user_credentials (
    user_email text PRIMARY KEY,
    user_id UUID,
    password TEXT
);
"""
)

session.execute(
    """
    CREATE TABLE IF NOT EXISTS langchain_agent_test.users (
    id UUID PRIMARY KEY,
    name TEXT,
    email TEXT
);"""
)

session.execute(
    """
    CREATE TABLE IF NOT EXISTS langchain_agent_test.user_videos ( 
    user_id UUID,
    video_id UUID,
    title TEXT,
    description TEXT,
    PRIMARY KEY (user_id, video_id)
);
"""
)

user_id = "522b1fe2-2e36-4cef-a667-cd4237d08b89"
video_id = "27066014-bad7-9f58-5a30-f63fe03718f6"

session.execute(
    f"""
    INSERT INTO langchain_agent_test.user_credentials (user_id, user_email) 
    VALUES ({user_id}, 'patrick@datastax.com');
"""
)

session.execute(
    f"""
    INSERT INTO langchain_agent_test.users (id, name, email) 
    VALUES ({user_id}, 'Patrick McFadin', 'patrick@datastax.com');
"""
)

session.execute(
    f"""
    INSERT INTO langchain_agent_test.user_videos (user_id, video_id, title)
    VALUES ({user_id}, {video_id}, 'Use Langflow to Build a LangChain LLM Application in 5 Minutes');
"""
)

session.set_keyspace("langchain_agent_test")

In [5]:
# Create a CassandraDatabase instance
# Uses the cassio session to connect to the database
db = CassandraDatabase()

In [6]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
toolkit = CassandraDatabaseToolkit(db=db)

tools = toolkit.get_tools()

print("Available tools:")
for tool in tools:
    print(tool.name + "\t- " + tool.description)

Available tools:
cassandra_db_schema	- 
    Input to this tool is a keyspace name, output is a table description 
    of Apache Cassandra tables.
    If the query is not correct, an error message will be returned.
    If an error is returned, report back to the user that the keyspace 
    doesn't exist and stop.
    
cassandra_db_query	- 
    Execute a CQL query against the database and get back the result.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    
cassandra_db_select_table_data	- 
    Tool for getting data from a table in an Apache Cassandra database. 
    Use the WHERE clause to specify the predicate for the query that uses the 
    primary key. A blank predicate will return all rows. Avoid this if possible. 
    Use the limit to specify the number of rows to return. A blank limit will 
    return all rows.
    


In [7]:
prompt = hub.pull("hwchase17/openai-tools-agent")

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [9]:
input = (
    QUERY_PATH_PROMPT
    + "\n\nHere is your task: Find all the videos that the user with the email address 'patrick@datastax.com' has uploaded to the langchain_agent_test keyspace."
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input})

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `cassandra_db_schema` with `{'keyspace': 'langchain_agent_test'}`


Table Name: user_credentials
- Keyspace: langchain_agent_test
- Columns
  - password (text)
  - user_email (text)
  - user_id (uuid)
- Partition Keys: (user_email)
- Clustering Keys: 

Table Name: user_videos
- Keyspace: langchain_agent_test
- Columns
  - description (text)
  - title (text)
  - user_id (uuid)
  - video_id (uuid)
- Partition Keys: (user_id)
- Clustering Keys: (video_id asc)


Table Name: users
- Keyspace: langchain_agent_test
- Columns
  - email (text)
  - id (uuid)
  - name (text)
- Partition Keys: (id)
- Clustering Keys: 


Invoking: `cassandra_db_select_table_data` with `{'keyspace': 'langchain_agent_test', 'table': 'user_credentials', 'predicate': "user_email = 'patrick@datastax.com'", 'limit': 1}`


Row(user_email='patrick@datastax.com', password=None, user_id=UUID('522b1fe2-2e36-4cef-a667-cd4237d08b89'))
Invoking: `cassandra_db_select_table_data` 